<a href="https://colab.research.google.com/github/LeoJanuario/Tech-Challenge-3/blob/main/techchalllenge3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtém dados da API

In [ ]:
import pandas as pd
import requests

#Coleta dados da API da AWS
url = 'https://6gt6hotdph.execute-api.us-east-1.amazonaws.com/Prod/GetCoinData'
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)

#Converter timestamps
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['data'] = df['timestamp'].dt.date

#Calcula médias diárias por moeda
df_cripto = df.groupby(['moeda', 'data']).agg({
    'preco': 'mean',
    'market_cap': 'mean',
    'volume': 'mean'
}).reset_index()

df_cripto = df_cripto.rename(columns={
    'preco': 'media_preco_diaria',
    'market_cap': 'media_market_cap_diaria',
    'volume': 'media_volume_diaria'
})

#Calcular preco_amanha (preço médio do dia seguinte)
df_cripto['preco_amanha'] = df_cripto.groupby('moeda')['media_preco_diaria'].shift(-1)

#Coluna alta_amanha (1 se preco_amanha > preco hoje)
df_cripto['alta_amanha'] = (df_cripto['preco_amanha'] > df_cripto['media_preco_diaria']).astype(int)


print(df_cripto.head())
print(f'\nTotal de registros: {len(df_cripto)}')


     moeda        data  media_preco_diaria  media_market_cap_diaria  \
0  bitcoin  2025-04-11       491100.064648             9.751007e+12   
1  bitcoin  2025-04-12       493188.916897             9.789977e+12   
2  bitcoin  2025-04-13       495617.880087             9.837817e+12   
3  bitcoin  2025-04-14       496275.704556             9.852151e+12   
4  bitcoin  2025-04-15       499219.055380             9.909780e+12   

   media_volume_diaria   preco_amanha  alta_amanha  
0         2.299170e+11  493188.916897            1  
1         1.779885e+11  495617.880087            1  
2         1.506487e+11  496275.704556            1  
3         1.858712e+11  499219.055380            1  
4         1.633237e+11  494756.554933            0  

Total de registros: 225


Treina o modelo CatBoostRegressor e CatBoostClassifier



In [ ]:
!pip install catboost
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

categorical_cols = ['moeda']
numerical_cols = ['media_preco_diaria', 'media_market_cap_diaria', 'media_volume_diaria']

df_model = df_cripto.dropna(subset=['preco_amanha', 'alta_amanha'])

X = df_model[categorical_cols + numerical_cols]
y_reg = df_model['preco_amanha']
y_class = df_model['alta_amanha']

#Treino e teste
X_train, X_test, y_reg_train, y_reg_test, y_class_train, y_class_test = train_test_split(
    X, y_reg, y_class, test_size=0.2, random_state=42
)

#CatBoost regressão e classificação
train_pool_reg = Pool(X_train, y_reg_train, cat_features=categorical_cols)
test_pool_reg = Pool(X_test, y_reg_test, cat_features=categorical_cols)

train_pool_class = Pool(X_train, y_class_train, cat_features=categorical_cols)
test_pool_class = Pool(X_test, y_class_test, cat_features=categorical_cols)

#Modelo de regressão
model_reg = CatBoostRegressor(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    loss_function='RMSE',
    eval_metric='RMSE',
    early_stopping_rounds=50,
    verbose=100,
    random_seed=42
)
model_reg.fit(train_pool_reg, eval_set=test_pool_reg, use_best_model=True)

#Modelo de classificação
model_class = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    eval_metric='Accuracy',
    early_stopping_rounds=50,
    verbose=100,
    random_seed=42
)
model_class.fit(train_pool_class, eval_set=test_pool_class, use_best_model=True)

y_reg_pred = model_reg.predict(X_test)
y_class_pred = model_class.predict(X_test)

#Conclusões
rmse = mean_squared_error(y_reg_test, y_reg_pred)
rmse = np.sqrt(rmse)

print(f"Regressão - RMSE: {rmse:.2f}")

accuracy = accuracy_score(y_class_test, y_class_pred)
print(f"Classificação - Acurácia: {accuracy:.2%}")

0:	learn: 206427.4173418	test: 172866.3746517	best: 172866.3746517 (0)	total: 1.63ms	remaining: 815ms
100:	learn: 5122.2784473	test: 10843.9296927	best: 10801.6288055 (98)	total: 115ms	remaining: 456ms
200:	learn: 2284.4082008	test: 9955.7651703	best: 9955.7651703 (200)	total: 233ms	remaining: 347ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 9930.641059
bestIteration = 222

Shrink model to first 223 iterations.
0:	learn: 0.6136364	test: 0.5000000	best: 0.5000000 (0)	total: 1.85ms	remaining: 923ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.5
bestIteration = 0

Shrink model to first 1 iterations.
Regressão - RMSE: 9930.64
Classificação - Acurácia: 50.00%


Previsão

In [ ]:
def prever_moeda(moeda_nome, df, model_reg, model_class):

    #Filtra o dataframe para pegar os dados mais recentes da moeda
    df_moeda = df[df['moeda'].str.lower() == moeda_nome.lower()].sort_values('data')

    if df_moeda.empty:
        return f"Moeda '{moeda_nome}' não encontrada no dataset."

    #Última linha do histórico
    ultima_linha = df_moeda.iloc[-1]
    preco_atual = ultima_linha['media_preco_diaria']

    #Monta o input para o modelo
    X_novo = pd.DataFrame([{
        'moeda': ultima_linha['moeda'],
        'media_preco_diaria': ultima_linha['media_preco_diaria'],
        'media_market_cap_diaria': ultima_linha['media_market_cap_diaria'],
        'media_volume_diaria': ultima_linha['media_volume_diaria']
    }])

    #Previsão
    preco_estimado = model_reg.predict(X_novo)[0]
    tendencia_pred = model_class.predict(X_novo)[0]

    #Corrige a tendência com base no valor estimado vs atual
    if preco_estimado > preco_atual:
        tendencia_str = 'Subir'
        variacao = ((preco_estimado - preco_atual) / preco_atual) * 100
    else:
        tendencia_str = 'Descer'
        variacao = ((preco_estimado - preco_atual) / preco_atual) * 100

    return (
        f"A tendência da '{moeda_nome}' é {tendencia_str}. "
        f"Estimativa de preço: R$ {preco_estimado:,.2f} "
        f"({variacao:+.2f}%)"
    )


Resultados

In [ ]:
from datetime import timedelta
import pandas as pd

#Bitcoin
df_btc = df_cripto[df_cripto['moeda'].str.lower() == 'bitcoin'].copy()
df_btc['data'] = pd.to_datetime(df_btc['data'])
df_btc = df_btc.sort_values('data')

ultima_linha = df_btc.iloc[-1]
data_ultima_disponivel = ultima_linha['data'].date()
preco_medio_ultimo_dia = ultima_linha['media_preco_diaria']
data_previsao = data_ultima_disponivel + timedelta(days=1)

print(f"Último dado disponível {data_ultima_disponivel}: Preço médio = R$ {preco_medio_ultimo_dia:,.2f}")

moeda = 'bitcoin'
resultado = prever_moeda(moeda, df_cripto, model_reg, model_class)
print(f"Previsão para o dia: {data_previsao.strftime('%Y-%m-%d')}: {resultado}")


Último dado disponível 2025-05-25: Preço médio = R$ 607,959.22
Previsão para o dia: 2025-05-26: A tendência da 'bitcoin' é Descer. Estimativa de preço: R$ 596,715.36 (-1.85%)
